### Load the relevant paths and modules

In [1]:
# set the auto-reload for the cells with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import os
import sys

# get source and data directories
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
data_dir = os.path.join(os.getcwd(), os.pardir, 'data')

# set the source data directory as the current path
src_data_dir = os.path.join(src_dir, 'data')
sys.path.append(src_data_dir)

In [3]:
# load the web scraping modules
%aimport feedback
import feedback as fb

In [4]:
# load the database modules
%aimport database
import database as db
import sqlite3

In [5]:
%aimport clean
import clean as cl

### Clean the existing csv data files

In [30]:
#audit movie financial csv file
cl.audit_imdb_data(data_dir)

# audit movie ratings csv file
cl.audit_additional_data(data_dir)

### Insert movie details data into SQL database

In [85]:
movie_db = os.path.join(data_dir, 'processed/movie_hype.db')
db.create_file(movie_db)

In [86]:
connection = sqlite3.connect(db.DB_FILE)
connection.text_factory = str
db.clear_tables(connection)

clearing tables...
done!


In [87]:
db.create_tables(connection)

creating tables...
done!


In [88]:
finance_csv = os.path.join(data_dir, 'processed/movie_finance.csv')
ratings_csv = os.path.join(data_dir, 'processed/movie_ratings.csv')
db.insert_finance_data(finance_csv, connection)
db.insert_ratings_data(ratings_csv, connection)

('inserting ', '/Users/dmoton/projects/data_projects/movie-hype/notebooks/../data/processed/movie_finance.csv', 'into', '/Users/dmoton/projects/data_projects/movie-hype/notebooks/../data/processed/movie_hype.db', '...')
done!
('inserting ', '/Users/dmoton/projects/data_projects/movie-hype/notebooks/../data/processed/movie_ratings.csv', 'into', '/Users/dmoton/projects/data_projects/movie-hype/notebooks/../data/processed/movie_hype.db', '...')
done!


### Scrape movie data and place in SQL database

In [17]:
# get a list of movies for a particular year
years = ['2012', '2013', '2014', '2015', '2016']
movie_data = []

# get a list of movie urls for each desired year
movie_urls = fb.get_movie_urls(years)
print(len(movie_urls))

getting movie urls... 
done!
1351


In [20]:
movie_data = fb.scrape(movie_urls)

scraping data from movie website...
done!


In [21]:
movie_data

[{'interest': 6.8, 'movie_title': u'brave'},
 {'interest': 3.98, 'movie_title': u'frankenweenie'},
 {'interest': 7.32, 'movie_title': u'chimpanzee'},
 {'interest': 1.11, 'movie_title': u'piranha 3dd'},
 {'interest': 5.54, 'movie_title': u'life of pi'},
 {'interest': 3.14, 'movie_title': u'wreckit ralph'},
 {'interest': 6.27, 'movie_title': u'the secret world of arrietty'},
 {'interest': 2.84, 'movie_title': u'finding nemo 3d'},
 {'interest': 9.22, 'movie_title': u'the dark knight rises'},
 {'interest': 3.1, 'movie_title': u'to kill a mockingbird'},
 {'interest': 1.22, 'movie_title': u'raiders of the lost ark'},
 {'interest': 28.03, 'movie_title': u'one for the money'},
 {'interest': 14.74, 'movie_title': u'think like a man'},
 {'interest': 5.04, 'movie_title': u'red dawn'},
 {'interest': 3.61, 'movie_title': u'the amazing spiderman'},
 {'interest': 27.93, 'movie_title': u'underworld awakening'},
 {'interest': 4.74, 'movie_title': u'wrath of the titans'},
 {'interest': 5.83, 'movie_titl

In [22]:
len(movie_data)

1351

In [23]:
interim_data_dir = os.path.join(data_dir, 'processed')
fb.convert_to_csv(movie_data, interim_data_dir)

creating /Users/dmoton/projects/data_projects/movie-hype/notebooks/../data/processed/interest_data.csv...
done!
